In [1]:
import random
import pandas as pd

import numpy as np

from keras.layers import Input, Conv2D, Dense, concatenate
from keras.models import Model

# Configuration

In [2]:
UPPER_BOUND = 20
N_DATAPOINTS = 100000

TRAINING_SET_SIZE = 0.7 # %
BATCH_SIZE = 1024
EPOCHS = 100

# Data

In [3]:
possible_labels = [1, 2]

values_1 = []
labels_1 = []

values_2 = []
labels_2 = []

for i in range(0, N_DATAPOINTS):
    values_1.append([random.randint(0, UPPER_BOUND)])
    rnd_label_index = random.randint(0, len(possible_labels) - 1)
    label = possible_labels[rnd_label_index]
    labels_1.append(label)

for i in range(0, N_DATAPOINTS):
    values_2.append([random.randint(0, UPPER_BOUND)])
    rnd_label_index = random.randint(0, len(possible_labels) - 1)
    label = possible_labels[rnd_label_index]
    labels_2.append(label)
    
values_1 = np.array(values_1)
labels_1 = np.array(labels_1)
values_2 = np.array(values_2)
labels_2 = np.array(labels_2)

In [4]:
values_1

array([[14],
       [ 1],
       [13],
       ...,
       [19],
       [ 0],
       [14]])

In [5]:
labels_1

array([1, 2, 2, ..., 2, 2, 1])

## Training set, Test set

In [6]:
# first dataset
datapoints_1 = len(values_1)

train_datapoints_1 = int(datapoints_1 * TRAINING_SET_SIZE)
test_datapoints_1 = datapoints_1 - train_datapoints_1

train_index_1 = np.random.choice(range(0, datapoints_1), size=train_datapoints_1, replace=False)
test_index_1 = list(set(range(0, datapoints_1)) - set(train_index_1))

X1_train = values_1[train_index_1]
X1_test = values_1[test_index_1]

Y1_train = labels_1[train_index_1]
Y1_test = labels_1[test_index_1]

# second dataset
datapoints_2 = len(values_2)

train_datapoints_2 = int(datapoints_2 * TRAINING_SET_SIZE)
test_datapoints_2 = datapoints_2 - train_datapoints_2

train_index_2 = np.random.choice(range(0, datapoints_2), size=train_datapoints_2, replace=False)
test_index_2 = list(set(range(0, datapoints_2)) - set(train_index_2))

X2_train = values_2[train_index_2]
X2_test = values_2[test_index_2]

Y2_train = labels_2[train_index_2]
Y2_test = labels_2[test_index_2]

# Neural Network with multiple input

In [7]:
input_1_shape = list(X1_train.shape[1:])
input_2_shape = list(X2_train.shape[1:])

print(input_1_shape)
print(input_2_shape)

[1]
[1]


In [8]:
import tensorflow as tf
from tensorflow import keras

in1 = keras.Input(shape=input_1_shape, name="IQ")
in2 = keras.Input(shape=input_2_shape, name="MP")

x = keras.layers.concatenate([in1, in2])

dense = keras.layers.Dense(256, kernel_initializer='he_normal', name="dense")(x)
output = keras.layers.Dense(1, kernel_initializer='he_normal', name="output")(dense)

model = keras.Model(inputs=[in1, in2], outputs=[output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
IQ (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
MP (InputLayer)                 [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           IQ[0][0]                         
                                                                 MP[0][0]                         
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          768         concatenate[0][0]            

In [9]:
model.compile(
    optimizer='adam', 
    loss='mean_squared_error',
)

In [10]:
model.fit(
    (X1_train, X2_train),
    Y1_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: 11.9698
Epoch 2/100
69/69 [==============================] - 0s 2ms/step - loss: 0.3199
Epoch 3/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2596
Epoch 4/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2528
Epoch 5/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2533
Epoch 6/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2516
Epoch 7/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2537
Epoch 8/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2548
Epoch 9/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2544
Epoch 10/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2539
Epoch 11/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2548
Epoch 12/100
69/69 [==============================] - 0s 2ms/step - loss: 0.2579
Epoch 13/100
69/69 [================

In [11]:
score = model.evaluate((X1_test, X2_test), Y1_test)

938/938 [==============================] - 1s 974us/step - loss: 0.2523


In [12]:
model.predict((X1_train, X2_train))

array([[1.5513843],
       [1.542149 ],
       [1.5359277],
       ...,
       [1.5559973],
       [1.5151657],
       [1.5216993]], dtype=float32)

In [13]:
Y1_test

array([2, 1, 2, ..., 2, 2, 1])